# 误差缓解及其实现

参考文献

[] K. Temme, S. Bravyi and J. M. Gambetta. Error mitigation for short-depth quantum circuits[J]. Phys. Rev. Lett. 119, 180509, 2017

[] A. Kandala, K. Temme, A. D. Córcoles, A. Mezzacapo, J. M. Chow and J. M. Gambetta. Error mitigation extends the computational reach of a noisy quantum processor[J]. Nature 2019 Vol. 567 Issue 7749 Pages 491-495




## Error mitigation for short-depth quantum circuits

开放系统的演化可以用以下公式描述:
\begin{equation}
\frac{\partial}{\partial t} \rho(t)=-i[K(t), \rho(t)]+\lambda \mathcal{L}[\rho(t)]
\end{equation}
观测量$A$在末态$ \rho_\lambda(T)$真实期望值为:
\begin{equation}
E_K(\lambda)=tr[A\rho_\lambda(T)]
\end{equation}

其中 $E_K(\lambda)$ 可以展开为$\lambda$的多项式:
\begin{equation}
E_{K}(\lambda)=E^{*} +\sum_{k=1}^{n} a_{k} \lambda^{k}+R_{n+1}(\lambda, \mathcal{L}, T)
\end{equation}
$E^{*}$ 是不含噪声的期望值.$a_k$是只和模型有关的参数. $R_{n+1}(\lambda, \mathcal{L}, T)$是展开式的留数且满足:
\begin{equation}
|R_{n+1}(\lambda, \mathcal{L}, T)|\leq \|A\|\frac{l_{n+1}(\lambda T)^{n+1}}{(n+1!)}
\end{equation}
我们想要得到的是对$E_K(\lambda)$进行外推得到的无噪声值$E^* =\lim_{\lambda\rightarrow 0}E_K(\lambda)$. 但实际上, 体系的误差系数$\lambda$往往是固定的, 我们没办法通过改变$\lambda$来外推得到$E^{*}$. 有一些巧妙的办法可以帮我们解决这一难题.

### Richardson's deferred approach to the limit

假设可以在噪声大小为$\lambda_j,j=0,1,\dots,n$时运行线路得到实验结果 $\hat{E}_K(\lambda)=E_K(\lambda)+\delta_j$, 其中$\lambda_j=c_j\lambda$, $\delta_j$源于有限采样误差. 定义以下估计量:
\begin{equation}
\hat{E}_K^n(\lambda)=\sum_{j=0}^{n}\gamma_j \hat{E}_K(c_j\lambda).\label{eq:e1}
\end{equation}
我们要求系数$\gamma_j$满足以下方程:
\begin{equation}
\sum_{j=0}^{n}\gamma_j =1\quad  \text{and}\quad \sum_{j=0}^{n}\gamma_jc_j^k =0,\quad\text{for} \quad k=1,\dots,n
\end{equation}
实际上, 该方程可以被有效求解, 其系数满足:
$$ \gamma_j=\prod_{m\neq j}c_m(c_j-c_m)^{-1}$$
在此定义下, 方程>e1 能以最多$\mathcal{O}(\lambda^{n+1})$的误差近似无噪声值$E^*$.

### Probabilistic error cancellation

含噪声线路的期望值为: $$E(\boldsymbol{\alpha})=\operatorname{tr}\left[A \mathcal{O}_{\alpha}\left(|0\rangle\left\langle\left. 0\right|^{\otimes n}\right)\right] .\right.$$
理想的无噪声期望值为: $$E^*(\boldsymbol{\beta})=\operatorname{tr}\left[A \mathcal{U}_{\beta}\left(|0\rangle\left\langle\left. 0\right|^{\otimes n}\right)\right] .\right.$$
**核心**:用噪声线路的线性组合表示理想的结果:
$$\mathcal{U}_{\beta}=\gamma_{\beta} \sum_{\alpha \in \Omega_{L}} P_{\beta}(\boldsymbol{\alpha}) \sigma_{\beta}(\boldsymbol{\alpha}) \mathcal{O}_{\boldsymbol{\alpha}}$$
上式被称为**准概率表示(quasiprobability representation)**,简写为**QPR**. 这样的话, 就得到理想的期望值为:
$$E^*(\beta)=\gamma_{\beta} \sum_{\alpha \in \Omega_{L}} P_{\beta}(\boldsymbol{\alpha}) \sigma_{\beta}(\boldsymbol{\alpha}) E(\boldsymbol{\alpha})$$

那么下面的问题就变成了:
1. 是否存在这样的QPR;
2. 能否构建这样的QPR;
3. 构建QPR的代价多大.


解决思路:
1. 第一个问题.参照原文.存在性略.
2. 第二个问题实际上这就转化为一个线性规划问题:
\begin{equation}
\textbf{minimize} \sum_{\alpha=1}^p\mu_\alpha
\end{equation}

\begin{equation}
\textbf{subject to} 
\left\{\begin{array}{ll}
\eta_\alpha &\leq \mu_\alpha\\
-\eta_\alpha&\leq \mu_\alpha \\
\mathcal{U}_\beta &=\sum_{\alpha=1}^p \eta_\alpha \mathcal{O}_\alpha
\end{array}\right.
\end{equation}

那么最优情况下一定满足$\mu_\alpha=|\eta_\alpha|$对所有$\alpha$都成立.定义$\gamma_\beta=\sum_{\alpha=1}^p\mu_\alpha$,$P_\beta(\alpha)=\mu_\alpha/\gamma_\beta$, 以及$\sigma_\beta(\alpha)=sgn(\eta_\alpha)$, 就可以得到QPR表示

$$\mathcal{U}_{\beta}=\gamma_{\beta} \sum_{\alpha=1}^p P_{\beta}(\alpha) \sigma_{\beta}(\alpha) \mathcal{O}_{\alpha}$$

单个门线路可以按照上述过程构建, 那么 gate-wise QPRs 的构建只需要令
\begin{equation}
\begin{aligned}
\gamma_{\boldsymbol{\beta}}&=\gamma_{\beta_1}\cdots \gamma_{\beta_L}\\
P_{\boldsymbol{\beta}}(\boldsymbol{\alpha})&=P_{\beta_1}(\alpha_1)\cdots P_{\beta_L}(\alpha_L)\\
\sigma_{\boldsymbol{\beta}}(\boldsymbol{\alpha})&=\sigma_{\beta_1}(\alpha_1)\cdots \sigma_{\alpha_L}
\end{aligned}
\end{equation}

可以限制量子操作最多作用在两个量子比特上, 那么这样的操作可以用 $16\times 16$ 的矩阵来表示, 其中每个矩阵元由相应的 Pauli 基唯一确定. 而解这样的线性规划问题, 时间复杂度是常数量级的($\mathcal{O}(1)$). 而理想门的量子操作基的大小为 $\mathcal{O}(n^2)$, 所以 QPRs 能在时间 $\mathcal{O}(n^2)$ 内计算得到.

3. 第三个问题需要考虑蒙特卡洛采样的代价.
首先注意到 $\langle x|A|x\rangle$ 是 $E(\boldsymbol{\alpha})$ 的无偏估计.

因为 $E_{|x\rangle}[\langle x|A|x\rangle]=E_{|x\rangle}[\operatorname{Tr}\left(A|x\rangle \langle x|\right)]$, 因为$|x\rangle$ 从噪声末态$\mathcal{O}_{\alpha}\left(|0\rangle \langle 0|^{\otimes n}\right)$中采样得到, 所以$|x\rangle \langle x|$ 的期望值就等于 $\mathcal{O}_{\alpha}\left(|0\rangle \langle 0|^{\otimes n}\right)$. 所以,

$$E_{|x\rangle}[\langle x|A|x\rangle]=\operatorname{tr}\left[A \mathcal{O}_{\alpha}\left(|0\rangle\langle\left. 0\right|^{\otimes n}\right)\right] =E(\boldsymbol{\alpha})$$
即, $\langle x|A|x\rangle$ 是 $E(\boldsymbol{\alpha})$ 的无偏估计量.

再注意到 $\langle x|A|x\rangle$ 估计$E(\boldsymbol{\alpha})$ 的方差为 $\mathcal{O}(1)$, 这是因为
$$V(\langle x|A|x\rangle)=E_{|x\rangle}[\langle x|A|x\rangle^2]-\left( E_{|x\rangle}[\langle x|A|x\rangle]\right)^2$$首先是大于0的, 其次第一项的值根据$\|A\|\leq 1$一定是小于等于1的. ($\mathcal{O}(\cdot)$的定义是小于或等于)

那么可以得到, $\gamma_\beta\sigma_\beta\langle x|A|x\rangle$ 是理想期望值 $E^*(\boldsymbol{\beta})$ 的无偏估计, 方差为 $\mathcal{O}(\gamma^2_\beta)$. 所以给定任意的精度 $\delta$, 我们可以通过蒙特卡洛方法在给定精度内估计理想的期望值 $E^*(\boldsymbol{\beta})$, 采样次数的根方与 $\gamma$ 成正比, 与精度 $\delta$ 成反比:
$$\sqrt{M}\propto \frac{\gamma_\beta}{\delta},$$

则根据分布 $P_\beta(\alpha)$ 生成 $M$ 个采样: $\alpha^1,\dots,\alpha^M\in \Omega_L$, 并且根据 Hoeffding 不等式, $\hat{E}(\beta)$ 可以 以误差 $\mathcal{O}(\delta)$ 近似 $E^*(\boldsymbol{\beta})$:
$$\hat{E}(\beta)=\frac{\gamma_\beta}{M}\sum_{a=1}^M \sigma_\beta(\alpha^a)\langle x^a|A|x^a\rangle.$$

#### Depolarizing noise cancellation

定义退极化噪声信道$$\mathcal{D}_k(\rho)=\epsilon \frac{I}{2^k}+(1-\epsilon)\rho$$

定义k比特含噪声门为$\mathcal{D}_k\mathcal{U}$.
定义噪声基为$\mathcal{O}_\alpha=\mathcal{D}_k\mathcal{P}\mathcal{U}$, 其中$k=1,2$.

##### 单比特门

已知线性规划等式条件为: $\mathcal{U}_\beta =\sum_{\alpha=1}^p \eta_\alpha \mathcal{O}_\alpha$, 其中噪声门形式已经被我们指定, 即退极化噪声下的理想门$ \mathcal{O}_\alpha=\mathcal{D}_1\mathcal{P}\mathcal{U}_\beta$.
那么我们可以得到:

$$ \mathcal{U}_\beta =\sum_{\alpha=1}^p \eta_\alpha \mathcal{D}_1\mathcal{P}\mathcal{U}_\beta$$
右乘上$\mathcal{U}^{-1}_\beta$并左乘上$\mathcal{D}_1^{-1}$, 得到:

$$\mathcal{D}_1^{-1} =\sum_{\alpha=1}^p \eta_\alpha \mathcal{P}_\alpha$$

并且, 单比特情况下, $p=4$, $\mathcal{P}$的基为$\{\mathcal{I},\mathcal{X},\mathcal{Y},\mathcal{Z}\}$, 遂得到:

$$\mathcal{D}_1^{-1}=\eta_1\mathcal{I}+\eta_2\mathcal{X}+\eta_3\mathcal{Y}+\eta_4\mathcal{Z}$$

我们注意到恒等式:
$$I=\frac{1}{2}\sum_{P}P\rho P $$
其中$P\in\{I,X,Y,Z\}$, 所以退极化信道可以写为:
$$ \mathcal{D}_1(\rho)=(1-\frac{3\epsilon}{4})\rho+\frac{\epsilon}{4}(X\rho X+Y\rho Y+Z\rho Y)$$

从而得到:

\begin{equation}
\begin{aligned}
\rho= \mathcal{D}_1^{-1}\mathcal{D}_1(\rho)=(\eta_1\mathcal{I}+\eta_2\mathcal{X}+\eta_3\mathcal{Y}+\eta_4\mathcal{Z})\left((1-\frac{3\epsilon}{4})\rho+\frac{\epsilon}{4}(X\rho X+Y\rho Y+Z\rho Y)\right) 
\end{aligned}
\end{equation}
可以计算得到:
$$\eta_1=1+\frac{3\epsilon}{4(1-\epsilon)}\quad \text{and}\quad \eta_\alpha=-\frac{\epsilon}{4(1-\epsilon)}\quad\text{for}\quad \alpha=2,3,4$$

$$\gamma_\beta=\sum_\alpha|\eta_\alpha|=\frac{1+\epsilon/2}{1-\epsilon}$$

##### 两比特门

类似地, 得到两比特退极化信道的表达式为:
$$ \mathcal{D}_2(\rho)=\left(1-\frac{15\epsilon}{16}\right)\rho+\frac{\epsilon}{16}\sum_{i=1}^{16} P_i\rho P_i$$
其逆可以由两比特Pauli信道的线性组合表示
$$\mathcal{D}_2^{-1} =\sum_{\alpha=1}^{16} \eta_\alpha \mathcal{P}_\alpha$$

可以计算得到:
$$\eta_1=1+\frac{15\epsilon}{16(1-\epsilon)}\quad \text{and}\quad \eta_\alpha=-\frac{\epsilon}{16(1-\epsilon)}\quad\text{for}\quad \alpha=2,3,\dots,16$$

$$\gamma_\beta=\sum_\alpha|\eta_\alpha|=\frac{1+7\epsilon/8}{1-\epsilon}$$

##### 构建噪声线路集合

1. 从理想无噪声线路开始, $\mathcal{O}_\alpha=\mathcal{U}_\beta$.
2. 每个单比特门之后以概率 $p_1=\epsilon/(4+2\epsilon)$ 添加Pauli $X,Y,Z$.单门以概率 $1-3p_1$ 保持不变. 
3. 每个双门后以概率 $p_2=\epsilon/(16+14\epsilon)$ 添加两比特Pauli门 $II,IX,\dots,ZZ$. 双门以概率 $1-15p_2$ 保持不变.

经过以上操作, 即得到$ \mathcal{O}_\alpha=\mathcal{D}\mathcal{P}\mathcal{U}_\beta$的形式.

4. 然后再在含噪声的量子计算机上运行, 记录读出的字符串$x$, 经过 $M$ 次的采样, 可以用 $\hat{E}(\beta)=\frac{\gamma_\beta}{M}\sum_{a=1}^M \sigma_\beta(\alpha^a)\langle x^a|A|x^a\rangle$ 来近似 $E^*(\boldsymbol{\beta})$.其中 $\sigma_\beta(\alpha^a)=(-1)^r$, $r$ 是向理想线路添加的Pauli算子的数量(例如在退极化噪声的情况下, 只有 $I$ 门或者 $II$ 门对应的 $\eta$ 是正值, 其他Pauli算子对应的 $\eta$ 均为负值). 

可以估计得到 
\begin{equation}
\begin{aligned}
\gamma_{\boldsymbol{\beta}}=&\left(\frac{1+\epsilon/2}{1-\epsilon} \right)^{L_1}\left(\frac{1+7\epsilon/8}{1-\epsilon} \right)^{L_2}\\
=&\left(1+\frac{1+3\epsilon/2}{1-\epsilon} \right)^{L_1}\left(1+\frac{1+15\epsilon/8}{1-\epsilon} \right)^{L_2}\\
\approx & \left(1+\frac{1+3\epsilon/2}{1-\epsilon}L_1 \right)\left(1+\frac{1+15\epsilon/8}{1-\epsilon}L_2 \right)\\
\approx & 1+\epsilon\left( 3L_1/2+15L_2/8\right)
\end{aligned}
\end{equation}

其中 $L_1$ 是单比特量子门的个数, $L_2$ 是两比特量子门的个数. 

(也就是说, 我们必须知道退极化噪声信道的退极化率, 才能有效地将$\mathcal{U}_\beta$用含噪声门$\mathcal{O}_\alpha$的线性组合表示出来.)

##### 随机噪声 Clifford+T 线路模拟

测试一个小集合的 Clifford+T线路, 噪声限制为退极化噪声. 选择理想门 $\mathcal{U}_{\boldsymbol{\beta}}$ 为$d$层线路, 每层要么由$n$个单比特门 $I,H,S,T$ 构成(奇数层), 要么由$n/2$个CNOT构成(偶数层). 随机选取500个这样的线路, 并令初态为$|+\rangle^{\otimes n}$. 对每个理想的线路 $\mathcal{U}_{\boldsymbol{\beta}}$, 我们选择观测量 $A$ 作为一个投影算子, 选取一半的计算基使期望值最小, 也即:

$$A=\sum_{x \in S}|x\rangle\langle x|, \quad S=\arg \max_{\substack{S \subseteq\{0,1\}^{n} \\|S|=2^{n-1}}} \sum_{x \in S}\left\langle x \left|\mathcal { U }_{\boldsymbol{\beta}} \left(|+\rangle\langle+|^{\otimes n}\right)|x\rangle\right.\right.$$

这样构造是为了 $E^*(\beta)\geq 1/2$ 对所有选定线路均成立. 因此可以得到overhead $\gamma_{\boldsymbol{\beta}}$仅仅取决于qubit个数和线路深度:

$$\gamma_{\boldsymbol{\beta}}=\left[\frac{1+\epsilon / 2}{1-\epsilon}\right]^{n d / 2} \cdot\left[\frac{1+7 \epsilon / 8}{1-\epsilon}\right]^{n d / 4}$$

这里用到一个优化的方法, 先把 $M$ 个采样分成 $K$ 组, 
$$M=\sum_{j=1}^K K_j$$
再定义一个随机变量
$$\hat{E}(\boldsymbol{\beta})=\gamma_{\boldsymbol{\beta}}\frac{1}{K}\sum_{j=1}^K \sigma_{\boldsymbol{\beta}}(\boldsymbol{\alpha}^j)\frac{1}{M_j}\sum_{a=1}^{M_j}\langle x^a_j|A|x^a_j\rangle.$$
其实就是相当于把字符串相同的结果一起算. 这样算的好处的, 新定义的估计值同样是理想结果的无偏估计, 而且其方差可以通过不同分组进行调整. 可以检验, 最优的分组方式为:
$$M_j\approx \frac{M\sigma_j}{\sum_{i=1}^K}\sigma_i,$$
其中 $\sigma_j^2=E(\alpha^j)-E(\alpha_j)^2$. 为了得到粗略的期望值, 每个线路$\alpha^j$ 必须运行一些次数. 结果看原文.

#### Amplitude Damping noise

该噪声和退极化噪声的不同之处在于, 其自身并不能模拟任意的理想线路, 因为$\mathcal{A}$ 不是一个幺元映射(unital map, 保持单位元不变的映射).首先定义振幅弛豫信道为:
$$\mathcal{A}(\rho)=A_0\rho A_0^\dagger+A_1\rho A_1^\dagger$$
其中
\begin{equation}
A_0=\begin{bmatrix}1&0\\ 0&\sqrt{1-\epsilon} \end{bmatrix} \text{   and   } A_1=\begin{bmatrix}0&\sqrt{\epsilon}\\ 0&0 \end{bmatrix}
\end{equation}
假设 $\mathcal{U}_{\beta}$ 是单比特理想门, 并且有QPR:
$$\mathcal{U}_{\beta}=\gamma_{\beta} \sum_{\alpha=1}^p P_{\beta}(\alpha) \sigma_{\beta}(\alpha) \mathcal{O}_{\alpha}$$
其中 $\mathcal{O}_{\alpha}=\mathcal{A}\mathcal{V}_{\alpha}$, $\mathcal{V}_{\alpha}$ 是某个幺正映射. 所以上述QPR可以写为:
$$\mathcal{A}^{-1}=\gamma_{\beta} \sum_{\alpha=1}^p P_{\beta}(\alpha) \sigma_{\beta}(\alpha) \mathcal{V}_{\alpha}\mathcal{U}_{\beta}^{-1}$$
一个非幺元映射不能写成若干幺元映射的组合, 所以上式无解. 为解决这个问题, 必须通过添加态制备来扩充噪声基. 给定一个单比特态 $|\psi\rangle \langle\psi|$, 定义一个态制备映射:
$$\mathcal{P}_{|\psi\rangle}(\rho)=\operatorname{Tr}(\rho)\cdot |\psi\rangle \langle\psi|.$$
令 $\mathcal{S}(\rho)=S\rho S^{-1}$为S门, 定义噪声基包含噪声态制备TPCP算符 $\mathcal{P}_{|\psi\rangle}(\rho)$, 其中 $|\psi\rangle=|+\rangle,|-\rangle,|0\rangle,|1\rangle$, 噪声单比特门 $\mathcal{A}\mathcal{U}_{\beta}$, $\mathcal{A}\mathcal{S}^{\pm 1}\mathcal{U}_{\beta}$, 以及噪声两比特门 $\mathcal{A}_c\mathcal{A}_t\mathcal{S}_c^y\mathcal{S}_t^z\mathcal{U}_{\text{cnot}}, y,z\in\{0,\pm 1\}$. 

Claim: 以上噪声门构成了噪声基 $\Omega$, 可以模拟任意的 Clifford+T 线路 $\boldsymbol{\beta}$, 其中开销 (overhead) 为:
$$ \gamma_{\boldsymbol{\beta}}\leq \gamma^{L_1+2L_2},\gamma=\frac{1+\epsilon}{1-\epsilon}.$$

证明的思路分为**单比特噪声门**和**两比特噪声门**.

##### 单比特噪声门

考虑一个单比特理想门 $\mathcal{U}_{\beta}\in \Gamma$. 定义噪声基为$\{\mathcal{O}_{1}=\mathcal{A}\mathcal{U}_{\beta},\mathcal{O}_{2}=\mathcal{A}\mathcal{S}\mathcal{U}_{\beta},\mathcal{O}_{2}=\mathcal{A}\mathcal{S}^{-1}\mathcal{U}_{\beta},\mathcal{O}_{4}=\mathcal{P}_{|0\rangle}\}$, 之所以 $\mathcal{P}_{|0\rangle}\in \Omega$ 是因为 $\mathcal{A}\mathcal{P}_{|0\rangle}=\mathcal{P}_{|0\rangle}$. 而且 $\mathcal{P}_{|0\rangle}=\mathcal{A}\mathcal{P}_{|0\rangle}\mathcal{U}_{\beta}$. 根据 $$\mathcal{U}_{\beta}=\sum_{\alpha=1}^p\eta_{\alpha}\mathcal{O}_{\alpha},$$
右乘上$\mathcal{U}^{-1}_\beta$并左乘上$\mathcal{A}^{-1}$, 得到:
$$\mathcal{A}^{-1}=\eta_1\mathcal{I}+\eta_2\mathcal{S}+\eta_3\mathcal{S}^{-1}+\eta_4\mathcal{P}_{|0\rangle}$$
可以计算得到最优的$\{\eta_i\}$ 为:
$$\eta_1=\frac{1}{\sqrt{1-\epsilon}},\eta_2=\eta_3=\frac{1-\sqrt{1-\epsilon}}{2(1-\epsilon)},\eta_4=-\frac{\epsilon}{1-\epsilon}$$
则 overhead即为前面提到的 $\gamma$.

##### 两比特噪声门

$$\mathcal{U}_{\text{cnot}}=\sum_{\alpha=1}^p\eta_{\alpha}\mathcal{O}_{\alpha}'\mathcal{O}_{\alpha},$$

其中 $\mathcal{O}_{\alpha}=\mathcal{A}_c\mathcal{A}_t\mathcal{S}_c^y\mathcal{S}_t^z\mathcal{U}_{\text{cnot}}\in\Omega$ 是一个合理的噪声操作, $\mathcal{O}_{\alpha}'$ 要么是单位算子要么是态制备映射 $\mathcal{P}_{|0\rangle}$, 且作用于控制比特或目标比特上. 并且注意到 $\mathcal{A}\mathcal{P}_{|0\rangle}=\mathcal{P}_{|0\rangle}\mathcal{A}$. 尽管 $\mathcal{O}_{\alpha}'\mathcal{O}_{\alpha}$ 可能不是一个合理的噪声基, 但是我们可以把 $\mathcal{O}_{\alpha}'$ 合并到CNOT之后的操作中. 而且单门 $I,S,T$ 可以被吸收进 态制备映射 $\mathcal{P}_{|0\rangle}$ 中, 因为它们作用于 $|0\rangle$ 上很平凡(无变化).只有当遇到 $H$ 时才会有一些变化, 这时态制备映射被替换为为 $\mathcal{P}_{|+\rangle}$. 

这时候 $\mathcal{P}_{|+\rangle}$ 可以是理想门, 即 $\mathcal{U}_{\beta}\equiv\mathcal{P}_{|+\rangle}$, 且可以用噪声基表示(why?):
$$ \mathcal{P}_{|+\rangle}=\eta_1\mathcal{A}\mathcal{P}_{|+\rangle}+\eta_2\mathcal{A}\mathcal{P}_{|-\rangle}+\eta_3\mathcal{A}\mathcal{P}_{|1\rangle}$$

可以解出最优解:

$$\eta_{1,2}=\pm \frac{1}{2}\left(\frac{1}{\sqrt{1-\epsilon}}\pm\frac{1-2\epsilon}{1-\epsilon} \right),\eta_3=\frac{\epsilon}{1-\epsilon}$$

其中 overhead 为 $\gamma'\leq \gamma$

##### 构建理想线路

理想线路 $\boldsymbol{\beta}$ 的QPR 构建过程:

1. 利用分解式 $\mathcal{U}_{\text{cnot}}=\sum_{\alpha=1}^p\eta_{\alpha}\mathcal{O}_{\alpha}'\mathcal{O}_{\alpha}$ 对 CNOT 门进行分解并将紧接着CNOT门的单门和态制备映射$\mathcal{P}_{|0\rangle} $进行归并(如果有的话). 那么CNOT门就可以用噪声门集合 $\Omega$ 中的门替换.剩下的门仅有单比特门 $\mathcal{U}_{\beta}\in \Gamma$ 以及态制备 $\mathcal{P}_{|+\rangle}$.

2. 每个理想的门操作被替换为其 QPR 表示. 注意到 CNOT 门的替换中 overhead 是 $\gamma^2$, 而单门和态制备 $\mathcal{P}_{|+\rangle}$ 每个仅仅贡献 $\gamma$, 所以总的 overhead 为:

$$ \gamma_{\boldsymbol{\beta}}\leq \gamma^{L_1+2L_2},\gamma=\frac{1+\epsilon}{1-\epsilon}.$$